In [1]:
import openai
import pandas as pd
import tqdm
import glob

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open("../apikey/apikey.txt", "r") as f:
    openai.api_key = f.readline().replace("\n", "")

In [3]:
def query_prompt(prompt, max_tokens=1000):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [4]:
import random

In [12]:
def get_df(fname):
    def f(categories):
        for cat in categories:
            for word in [
                "geology",
                "physics",
                "chemistry",
                "mathematical",
                "biology",
                "astronomy",
                "ecology",
                "genetics",
                "statistics",
                "theoretical"
            ]:
                if word.lower() in cat.lower():
                    return True
            
        return False
    
    def text_preprocess(text):
        return text.replace("====", "\n\n").replace("===", "\n\n").replace("==", "\n\n")

    def sep_n(text, n=600):
        try:
            text = text.split(" ")
            start_index = random.randint(0, len(text) - n)
            return " ".join(text[start_index:start_index+n])
        except:
            return None
    
    df = pd.read_parquet(fname)
    df_science = df[df["categories"].apply(f)]
    df_science["text"] = df_science["text"].apply(text_preprocess)
    df_science["text"] = df_science["text"].apply(sep_n)
    df_science = df_science[df_science["text"].notnull()]
    return df_science

In [13]:
files = glob.glob("../data/wikipedia/*.parquet")

In [14]:
import time

In [15]:
import pickle

In [16]:
import json

In [17]:
from datetime import datetime as dt
import os

In [18]:
texts = []

In [19]:
import traceback 
batch_size = 1

def make_prompt(series):
    prompt = f"""
You are a professional machine learning engineer who creates datasets for use in supervised learning of multiple choice questions and are very knowledgeable about science.
Please make a multiple-choice questions about below context.

Context:
{series['text']}

Attention:
- The output should be json format.
- Json format key is "prompt" as the question statement, "A," "B," "C," "D," and "E" as choices, "answer" as the answer choice (one of A through E).
"""
    return prompt

def f(series):
    try:
        if series["A"] != series["A"]:
            if type(series["answer"]) == dict:
                for key in ["A", "B", "C", "D", "E"]:
                    series[key] = series["choices"][key]
            elif type(series["answer"] == list):
                for i, key in enumerate(["A", "B", "C", "D", "E"]):
                    series[key] = series["choices"][i]
    except:
        return series
    return series

now_date = dt.now().strftime("%Y%m%d%H%M%S")

first = True
for file in files:
    if os.path.basename(file) in ["all.parquet"]:
        print(f"pass: {file}")
        continue
    df_science = get_df(file)
    
    for i in tqdm.tqdm(range(len(df_science)), desc=file):
        try:
            series = df_science.iloc[i]
            prompt = make_prompt(series)
            text = query_prompt(prompt)
            texts_json = json.loads(text)
            if first:
                print(texts_json)
                first = False
            if type(texts_json) == dict:
                texts_json["wiki_id"] = series["id"]
                texts_json["original_text"] = series["text"]
                texts.append(texts_json)
            else:
                for text_json in texts_json:
                    text_json["wiki_id"] = series["id"]
                    text_json["original_text"] = series["text"]
                    texts.append(text_json)
        except Exception as e:
            print(e)
            traceback.print_exc()
            print(text)
        if i % 20 == 0:
            df_texts = pd.DataFrame(texts)
            df_texts = df_texts.apply(f, axis=1)

            df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

../data/wikipedia/a.parquet:   0%|▏                                                                                                                                                                                                             | 1/1097 [00:05<1:42:04,  5.59s/it]

{'prompt': "According to the book 'A Brief History of Everyone Who Ever Lived', what did DNA analysis reveal about the lineage of European kings?", 'A': 'They were descendants of Charlemagne.', 'B': 'They were descendants of Neanderthals.', 'C': 'They practiced inbreeding.', 'D': 'They had a high prevalence of red hair.', 'E': 'They had a high prevalence of lactose intolerance.', 'answer': 'C'}


../data/wikipedia/a.parquet:   2%|███▌                                                                                                                                                                                                         | 19/1097 [01:35<1:26:25,  4.81s/it]Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/conda/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/opt/conda/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/opt/co

Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
{
  "prompt": "Who won the Golden Boot in the 2021–22 W-League season?",
  "A": "Sam Kerr",
  "B": "Fiona Worts",
  "C": "Hannah Keane",
  "D": "Emily Gielnik",
  "E": "Rachel Lowe",
  "answer": "B"
}


../data/wikipedia/a.parquet:   5%|██████████▎                                                                                                                                                                                                  | 55/1097 [09:58<1:36:48,  5.57s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_11445/598235146.py", line 44, in <module>
    text = query_prompt(prompt)
  File "/tmp/ipykernel_11445/72703211.py", line 2, in query_prompt
    response = openai.ChatCompletion.create(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
 

...

../data/wikipedia/w.parquet:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋      | 350/361 [26:33<00:58,  5.29s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_11445/598235146.py", line 45, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Invalid \escape: line 3 column 23 (char 116)
../data/wikipedia/w.parquet:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Invalid \escape: line 3 column 23 (char 116)
{
  "prompt": "What is the probability density function of the wrapped normal distribution?",
  "A": "f_{WN}(\theta;\mu,\sigma)=\frac{1}{\sigma \sqrt{2\pi}} \sum^{\infty}_{k=-\infty} \exp \left[\frac{-(\theta - \mu + 2\pi k)^2}{2 \sigma^2} \right]",
  "B": "f_{WN}(\theta;\mu,\sigma)=\frac{1}{2\pi}\sum_{n=-\infty}^\infty e^{-\sigma^2n^2/2+in(\theta-\mu)}",
  "C": "f_{WN}(\theta;\mu,\sigma)=\frac{1}{2\pi}\vartheta\left(\frac{\theta-\mu}{2\pi},\frac{i\sigma^2}{2\pi}\right)",
  "D": "f_{WN}(\theta;\mu,\sigma)=\frac{1}{2\pi}\prod_{n=1}^\infty (1-q^n)(1+q^{n-1/2}z)(1+q^{n-1/2}/z)",
  "E": "None of the above",
  "answer": "C"
}


../data/wikipedia/y.parquet:  37%|████████████████████████████████████████████████████████████████████████████▋                                                                                                                                    | 33/90 [02:26<03:45,  3.96s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_11445/598235146.py", line 45, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Invalid \escape: line 3 column 16 (char 143)
../data/wikipedia/y.parquet:  38%|██████████████████████████████████████████████████████████████████████████████▉                                                     

Invalid \escape: line 3 column 16 (char 143)
{
  "prompt": "Which of the following is a condition that the Yeoh model must satisfy for consistency with linear elasticity?",
  "A": "2C_1 = \mu",
  "B": "2C_1 = \mu\,",
  "C": "2C_1 = \mu\, (i e j)",
  "D": "2C_1 = \mu\, (i e j)",
  "E": "2C_1 = \mu\, (i e j)",
  "answer": "B"
}


../data/wikipedia/z.parquet: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [04:53<00:00,  4.51s/it]


In [20]:
df_texts = pd.DataFrame(texts)
df_texts = df_texts.apply(f, axis=1)

df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")